In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/all_sauce5.zip

In [ ]:
import numpy as np
import cv2
import os
import csv

from os import listdir
from os.path import isfile, join

In [ ]:
mypath = 'all_sauce/'
imagePaths = [mypath+f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
def hog(image_file):
    img = cv2.imread(image_file)
    if(len(img) <len(img[0])):
        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)


    img =cv2.resize(img, (64,128))

    #print(img.shape[0:2])

    magnitude_matrix = np.zeros(img.shape[:2])
    angle_matrix = np.zeros(img.shape[:2])
    #print(angle_matrix.shape)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ##cv2.imshow('img',img)
    #########################

    #find Magnitude and Angle
    Gx = np.zeros(img.shape[:2])
    Gy = np.zeros(img.shape[:2])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if(col-1 < 0):
                Gx[row][col] = int(img[row][col+1])-0
            elif(col+1 == img.shape[1]):
                Gx[row][col] = 0-int(img[row][col-1])
            else:
                Gx[row][col] = int(img[row][col+1])-int(img[row][col-1])
            if(row-1 < 0):
                Gy[row][col] = 0-int(img[row+1][col])
            elif(row+1 == img.shape[0]):
                Gy[row][col] = int(img[row-1][col])-0
            else:
                Gy[row][col] = int(img[row-1][col])-int(img[row+1][col])

    for row in range(len(img)):
        for col in range(len(img[0])):
            magnitude_matrix[row][col] = (Gx[row][col]**2+Gy[row][col]**2)**(1/2)
            Gy_diveide_by_Gx = 0
            if(Gx[row][col] == 0):
                Gy_diveide_by_Gx = 0
            else:
                Gy_diveide_by_Gx = Gy[row][col]/Gx[row][col]

            angle = np.arctan(Gy_diveide_by_Gx)*180/np.pi
            if(angle < 0 ):
                angle_matrix[row][col] = 180+angle
            else:
                angle_matrix[row][col] = angle
            #angle_matrix[row][col] = np.arctan(Gy[row][col]/Gx[row][col])*180/np.pi


    ##print(angle_matrix)
    ##for i in range(len(angle_matrix)):
    ##    print(angle_matrix[i])
    ##print(Gy)
    ##for i in range(len(Gy)):
    ##    print(Gx[i])
    ##max_angle = 0
    ##for row in range(len(img)):
    ##    for col in range(len(img[0])):
    ##        if(angle_matrix[row][col]>max_angle):
    ##            #max_angle = angle_matrix[row][col]
    ##print(max_angle)
    ##cv2.imshow('angle',Gx+Gy)

    ###############################################################
    #use magtitude and angle to histogram (use each 8*8 pixel to find 9 bin)
    kernal_for_bin = [8,8]
    row_span = len(img)//kernal_for_bin[0]
    col_span = len(img[0])//kernal_for_bin[1]
    range_bins = [0,20,40,60,80,100,120,140,160,180]
    #print('col_span = ',col_span)
    # each bin = 20 degree
    histogram_bins = []
    for num_row in range(row_span):
        histogram_bins.append([])
    #print(len(histogram_bins),len(histogram_bins[0]))
    for row_block in range(row_span):
        for col_block in range(col_span):
            bins = [0,0,0,0,0,0,0,0,0]
            for row in range(row_block*kernal_for_bin[0],row_block*kernal_for_bin[0]+kernal_for_bin[0]):
                for col in range(col_block*kernal_for_bin[1],col_block*kernal_for_bin[1]+kernal_for_bin[1]):
                    try:
                        bins_position = int(angle_matrix[row][col]//20)
                    except:
                        bins_position = 0
    ##                    print(angle_matrix[row][col])
    ##                    print("row = ",row," col = ",col)
                    bins_position_next = bins_position+1
                    bins_position_cal = magnitude_matrix[row][col]*(abs((range_bins[bins_position]-angle_matrix[row][col])/20))
                    bins_position_next_cal = magnitude_matrix[row][col]*(abs((range_bins[bins_position_next]-angle_matrix[row][col])/20))
                    if(bins_position+1 == 9):
                        bins_position_next = 0
                    bins[bins_position] = bins[bins_position]+bins_position_cal
                    bins[bins_position_next] = bins[bins_position_next]+bins_position_next_cal
            histogram_bins[row_block].append(bins)



    ##############################################################
    #use 4 9-bin = 36 (2*2 of 8*8) to normalize then we'll get feature from HOG
    #print(len(histogram_bins),len(histogram_bins[0]))
    hog_feature = []
    for row_block in range(row_span-1):
        for col_block in range(col_span-1):
            #print(row_block," ",col_block)
            size_of_4_bins = np.dot(np.array(histogram_bins[row_block][col_block]),np.array(histogram_bins[row_block][col_block]))
            size_of_4_bins = size_of_4_bins+ np.dot(np.array(histogram_bins[row_block+1][col_block]),np.array(histogram_bins[row_block+1][col_block]))
            size_of_4_bins = size_of_4_bins+ np.dot(np.array(histogram_bins[row_block][col_block+1]),np.array(histogram_bins[row_block][col_block+1]))
            size_of_4_bins = size_of_4_bins+ np.dot(np.array(histogram_bins[row_block+1][col_block+1]),np.array(histogram_bins[row_block+1][col_block+1]))
            size_of_4_bins = size_of_4_bins**(1/2)
            hog_feature = hog_feature+(np.array(histogram_bins[row_block][col_block])/size_of_4_bins).tolist()
            hog_feature = hog_feature+(np.array(histogram_bins[row_block+1][col_block])/size_of_4_bins).tolist()
            hog_feature = hog_feature+(np.array(histogram_bins[row_block][col_block+1])/size_of_4_bins).tolist()
            hog_feature = hog_feature+(np.array(histogram_bins[row_block+1][col_block+1])/size_of_4_bins).tolist()
    return hog_feature





In [ ]:
with open('img_csv.csv','w', newline='') as f_img:
    write_bins = csv.writer(f_img)
    for (i, imagePath) in enumerate(imagePaths):
        product_name = imagePath.split(".")[0].split("/")[1]
        # print(product_name)
        product_class = 0
        if (product_name == "maeyupa"):
            product_class = 1
        elif (product_name == "ocean"):
            product_class = 2
        elif (product_name == "zapnuer"):
            product_class = 3
        elif (product_name == "zapnuerchick"):
            product_class = 4
        hog_data = hog(imagePath)
        write_bins.writerow([product_class]+hog_data)
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(imagePaths)))
f_img.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:116: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in true_divide


[INFO] processed 1000/2718
[INFO] processed 2000/2718


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip drive/"My Drive"/korat_sauce/test_all_sauce4.zip

In [ ]:
mypath_test = 'test_all_sauce/'
imagePaths_test = [mypath_test+f for f in listdir(mypath_test) if isfile(join(mypath_test, f))]

In [ ]:
#######
with open('test_img_csv.csv','w', newline='') as f_img:
    write_bins = csv.writer(f_img)
    for (i, img_path) in enumerate(imagePaths_test):
      product_name = img_path.split(".")[0].split("/")[1]
      product_class = 0
      if (product_name == "maeyupa"):
          product_class = 1
      elif (product_name == "ocean"):
          product_class = 2
      elif (product_name == "zapnuer"):
          product_class = 3
      elif (product_name == "zapnuerchick"):
          product_class = 4
      hog_data = hog(img_path)
      write_bins.writerow([product_class]+hog_data)
      if i > 0 and i % 1000 == 0:
          print("[INFO] processed {}/{}".format(i, len(imagePaths)))
f_img.close()
      
  